In [1]:
import numpy as np
import pandas as pd
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch
import time

torch.manual_seed(1)

In [2]:
with open(r'C:\Users\nelso\OneDrive\Machine_Learning\Project\tweets.txt', encoding = 'utf8') as f:
    tweets = f.readlines()

del tweets[0]

In [3]:
with open(r'C:\Users\nelso\OneDrive\Machine_Learning\Project\tweets.txt', encoding = 'utf8') as f:
    content=list(set(f.read()))

In [4]:
vocab_size = len(content)
ix_to_char = {ix:char for ix, char in enumerate(content)}
char_to_ix = {char:ix for ix, char in enumerate(content)}

A few notes on building the neural network
- Sequence length should vary for each tweet. Should be length of tweet.
- Need to create an \<EOS> marker
- Embedding happens when putting input tweet into x tensor
- From embedding, x should be passed through LSTM
- Then need a hidden layer which should maybe spit out something the size of vocab?
- Use a softmax to get the probability
- NLLL to get the loss
- Backprop
- Run this through 2 for loops: for epoch and for tweet

In [5]:
epochs = 2
hidden_dim = 128
embed_size = 128 # This is going to probably get up to 128 or 256

losses = []
loss_function = nn.NLLLoss()
model = nn.LSTM(embed_size, hidden_dim, num_layers = 3)
optimizer = optim.SGD(model.parameters(), lr=0.1)

start = time.time()
for epoch in range(epochs):
    total_loss = 0
    for tweet in tweets:
        #Create the index for our tweet and our target
        tweet_x = tweet[:-1]
        context_size = len(tweet_x)
        tweet_id = torch.tensor([char_to_ix[c] for c in tweet_x], dtype=torch.long)
        target_tweet = tweet[1:]
        target_ix = torch.tensor([char_to_ix[c] for c in target_tweet], dtype=torch.long) 
        
        #Reset gradient for each iteration
        model.zero_grad()
        
        #Reset hidden layer
        hidden = (torch.zeros(3, 1, hidden_dim), torch.zeros(3, 1, hidden_dim))
        
        #Embed characters and forward pass through Relu and Softmax
        embed = nn.Embedding(vocab_size, embed_size)
        embeds = embed(tweet_id)
        output, hidden = model(embeds.view(len(tweet_id), 1, -1), hidden)
        lin_2 = nn.Linear(hidden_dim, vocab_size)
        out = F.relu(lin_2(output.view(context_size, -1)))
        log_probs = F.log_softmax(out, dim=1)
        
        #Compute the loss function
        loss = loss_function(log_probs, target_ix)
        
        #Backprop and Update Gradient
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
    print('Epoch', epoch, 'Completed in', time.time()-start)
    losses.append(total_loss)
print (losses)
        
        
        
        
        

KeyboardInterrupt: 

In [26]:
def generate_tweet(first):
    char = torch.tensor(char_to_ix[first], dtype = torch.long)
    hidden = (torch.zeros(3, 1, hidden_dim), torch.zeros(3, 1, hidden_dim))
    output_tweet = first
    letter = first
    
    #while letter != '/n':
    for i in range(20):
        embeds = embed(char)
        output, hidden = model(embeds.view(1, 1, -1), hidden)
        topv, topi = output.topk(1)
        topi = topi[0][0][0].item()
        letter = ix_to_char[topi]
        output_tweet += letter
        char = torch.tensor(char_to_ix[letter], dtype = torch.long)
        
    return output_tweet

In [28]:
generate_tweet('b')

'bs!!!!w!wRRR!!!!!!w!w'